In [1]:
import ee
import geopandas as gpd
import geemap
import geersd
import pandas as pd

from pph import opca

In [2]:
ee.Initialize()

In [3]:
# load MM images
gdf = gpd.read_file("../../scratch/moose_mnt_l8_34_25_100.geojson")
gdf.shape

(82, 13)

In [4]:
gdf_roi = gpd.read_file("../../scratch/moose_mountain_FT_OPC_analysis_AOI_7year_L8.shp")
gdf_roi.set_crs(4326, inplace=True)
aoi = ee.FeatureCollection(gdf_roi.__geo_interface__).geometry()

In [5]:
gdf_2017 = gdf[(gdf.Year <= 2024) & (gdf.CLOUD_COVER < 6)]


In [6]:
stack = None
for image in gdf_2017['system:id'].tolist():
    img = ee.Image(image).select('SR_B5')
    if stack is None:
        stack = img
    else:
        stack = stack.addBands(img)
    

In [7]:
pca = opca.PrincipalCompoent().compute(stack, aoi)

In [9]:
task = ee.batch.Export.image.toDrive(
    image=pca.clip(aoi),
    region=aoi,
    scale=30,
    folder='moose-mountain-pca',
    description="moose-mountain-pca",
    crs='EPSG:4326',
    maxPixels=1e13,
    skipEmptyTiles=True
)

task.start()